In [126]:
import numpy as np
import pandas as pd

from bokeh.layouts import layout, column, row
from bokeh.models import CustomJS, Slider, Button, Label, CDSView, CustomJSFilter
from bokeh.plotting import ColumnDataSource, figure
from bokeh.io import output_notebook, show

from datetime import date, timedelta

# Create example data

data can only be used with boke server, I need it different for javascript callbacks

In [ ]:
data = {}
my_date = date.fromisoformat('2020-01-01')
dates = [my_date + timedelta(days=num_days) for num_days in range(0, 10)]
for date in dates:
    x = np.array([i for i in range(1, 11)])
    y = np.random.randint(20, size=(10))
    df = pd.DataFrame({'x':x, 'y':y})
    data[date] = df

In [128]:
source = ColumnDataSource(data=data[dates[0]])
plot = figure(y_range=(0, 25), plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

def slider_update(attrname, old, new):
    date = dates[slider.value]
    source.data = data[date]

slider = Slider(start=0, end=len(dates) - 1, value=0, step=1, title="Date")
button = Button(label="Play", button_type="success")
slider.on_change('value', slider_update)

layout = row(plot, column(slider, button))

output_notebook()
show(layout)

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [134]:
data = pd.DataFrame()
my_date = date.fromisoformat('2020-01-01')
dates = [my_date + timedelta(days=num_days) for num_days in range(0, 10)]
for date in dates:
    d = np.full((10), str(date))
    x = np.array([i for i in range(1, 11)])
    y = np.random.randint(20, size=(10))
    df = pd.DataFrame({'d':d, 'x':x, 'y':y})
    data = data.append(df, ignore_index=True)

In [138]:
source = ColumnDataSource(data=data)
custom_filter = CustomJSFilter(code='''
var indices = [];
for (var i = 0; i < source.get_length(); i++){
    if (source.data['d'][i] == '2020-01-01'){
        indices.push(true);
    } else {
        indices.push(false);
    }
}
return indices;
''')
view = CDSView(source=source, filters=[custom_filter])

plot = figure(y_range=(0, 25), plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, view=view, line_width=3, line_alpha=0.6)

button_callback = CustomJS(args=dict(source=source, view=view),
                    code='''
    console.log(view)
''')

slider = Slider(start=0, end=len(dates) - 1, value=0, step=1, title="Date")
button = Button(label="Play", button_type="success")
#slider.on_change('value', slider_update)

button.js_on_click(button_callback)

layout = row(plot, column(slider, button))

output_notebook()
show(layout)

Loading BokehJS ...

ERROR:bokeh.core.validation.check:E-1024 (CDSVIEW_FILTERS_WITH_CONNECTED): CDSView filters are not compatible with glyphs with connected topology such as Line or Patch: GlyphRenderer(id=5057, glyph=Line(id='5055', ...), ...)


In [ ]:
source = ColumnDataSource(data=data)
custom_filter = CustomJSFilter(code='''
var indices = [];
for (var i = 0; i < source.get_length(); i++){
    if (source.data['d'][i] == '2020-01-01'){
        indices.push(true);
    } else {
        indices.push(false);
    }
}
return indices;
''')
view = CDSView(source=source, filters=[custom_filter])

plot = figure(y_range=(0, 25), plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, view=view, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source, slider=slider),
                    code="""
    const data = source.data;
    const s = slider.value;
    const x = data['x'];
    const y = data['y'];
    for (var i = 0; i < x.length; i++) {
        y[i] = y[i] * s;
    }
    source.change.emit();
""")

slider = Slider(start=0, end=len(dates) - 1, value=0, step=1, title="Date")
button = Button(label="Play", button_type="success")
#slider.on_change('value', slider_update)

layout = row(plot, column(slider, button))

output_notebook()
show(layout)

In [131]:
plot = figure(y_range=(0, 40), plot_width=400, plot_height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=1, end=3, value=1, step=.1, title="Slider")

callback = CustomJS(args=dict(source=source, slider=slider),
                    code="""
    const data = source.data;
    const s = slider.value;
    const x = data['x'];
    const y = data['y'];
    for (var i = 0; i < x.length; i++) {
        y[i] = y[i] * s;
    }
    source.change.emit();
""")

slider.js_on_change('value', callback)

button = Button(label="Play", button_type="success")

cb = CustomJS(args=dict(source=source),
                    code="""
    const data = source.data;
    const x = data['x'];
    const y = data['y'];
    for (var i = 0; i < x.length; i++) {
        y[i] = y[i] * 2;
    }
    source.change.emit();
""")
button.js_on_click(cb)

layout = row(
    plot,
    slider,
    button,
)

output_notebook()

show(layout)

Loading BokehJS ...